In [1]:
import pandas as pd
from PIL import Image
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm
from sklearn.model_selection import train_test_split

### Caricamento delle immagini in input in formato array numpy

In [2]:
# IMAGE_PATH_FOLDER = './Dati Sensori/Dataset Sensori Lavorato/1_M/1_M_1'
IMAGE_PATH_FOLDER = '../../Nuovi Dati/DatasetDistorti'

# Specificare solamente una tipologia di speckle
DISTORSIONI = ['DatasetBlur-s']
GRADI_DISTORSIONE = 5

HEIGHT = 512
WIDTH = 512

TARGET_SIZE = (WIDTH, HEIGHT)
CHANNELS = 1 # Scala di grigi

In [3]:
def get_sotto_cartelle(path):
    return [x[1] for x in os.walk(path)][0]

def get_immagini_cartella(path_cartella):
    immagini_cartella = []
    cartelle_soggetti = get_sotto_cartelle(path_cartella)
    for soggetto in tqdm(cartelle_soggetti):
        path_soggetto = f'{path_cartella}/{soggetto}'
        cartelle_acquisizioni = get_sotto_cartelle(path_soggetto)
        for acquisizione in cartelle_acquisizioni:
            path_acquisizione = f'{path_soggetto}/{acquisizione}'
            for filename in os.listdir(path_acquisizione):
                if (filename.endswith(".jpg")):    
                    path_file = f'{path_acquisizione}/{filename}'
                    immagine = get_immagine_da_path(path_file)
                    immagini_cartella.append(immagine)
    return immagini_cartella
                    
def get_immagine_da_path(path_file):
    image = Image.open(path_file).convert('L')
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    return normalized_image_array


In [4]:
image_list = []
label_list = []

for distorsione in DISTORSIONI:
    path_distorsione = f'{IMAGE_PATH_FOLDER}/{distorsione}'
    for grado_distorsione in range(GRADI_DISTORSIONE):
        label = 100 - float(20 * (grado_distorsione))
        path_cartella_grado = f'{path_distorsione}{grado_distorsione}'
        immagini_grado = get_immagini_cartella(path_cartella_grado)
        image_list.extend(immagini_grado)
        label_to_append = [label for x in range(len(immagini_grado))] 
        label_list.extend(label_to_append)
        

100%|██████████| 719/719 [00:03<00:00, 216.95it/s]


In [5]:
train_images = np.array(image_list)
train_weights = np.array(label_list)

In [6]:
X = train_images
y = train_weights

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  
])

2023-07-25 10:40:20.440775: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-25 10:40:20.443013: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


In [9]:
model.compile(optimizer='adam', loss='mae', metrics = ['mae'])

In [ ]:
model.fit(x=X_train, y=y_train, batch_size=1, epochs=20)

Epoch 1/20


2023-07-25 10:40:49.400782: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-25 10:40:49.868965: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2876/2876 [==============================] - 312s 107ms/step - loss: 8.7189 - mae: 8.7189
Epoch 2/20
2876/2876 [==============================] - 304s 106ms/step - loss: 4.8370 - mae: 4.8370
Epoch 3/20
2876/2876 [==============================] - 304s 106ms/step - loss: 4.0810 - mae: 4.0810
Epoch 4/20
2876/2876 [==============================] - 304s 106ms/step - loss: 3.5230 - mae: 3.5230
Epoch 5/20
2876/2876 [==============================] - 303s 105ms/step - loss: 3.1360 - mae: 3.1360
Epoch 6/20
2876/2876 [==============================] - 305s 106ms/step - loss: 2.8571 - mae: 2.8571
Epoch 7/20
2876/2876 [==============================] - 302s 105ms/step - loss: 2.6876 - mae: 2.6876
Epoch 8/20
2876/2876 [==============================] - 302s 105ms/step - loss: 2.7805 - mae: 2.7805
Epoch 9/20
2876/2876 [==============================] - 302s 105ms/step - loss: 2.2885 - mae: 2.2885
Epoch 10/20
2876/2876 [==============================] - 302s 105ms/step - loss: 2.5029 - mae: 2.5029


In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions_list = [i[0] for i in predictions] 

In [ ]:
clamp_pred = [min(100,i) for i in predictions_list]
clamp_pred = [max(0,i) for i in clamp_pred]

In [ ]:
max(abs(clamp_pred  - y_test))

In [ ]:
def avg(lst):
    return sum(lst) / len(lst)

avg(abs(clamp_pred - y_test))

In [ ]:
model.save('../../Modelli-v2/modello_pesi_inizializzati_gaussian_blur')